In [ ]:
# 导入相关包
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 加载数据和模型
dataset = Dataset.load_from_disk("../data/alpaca_data_zh/")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [ ]:
# 数据预处理
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenizer_dataset = dataset.map(process_func, remove_columns=dataset.column_names)
# tokenizer.decode(tokenizer_dataset[1]["input_ids"])
tokenizer.decode(list(filter(lambda x: x != -100, tokenizer_dataset[1]["labels"])))

In [ ]:
# 配置peft并应用到模型
from peft import get_peft_model, TaskType

# bitfit    将模型参数里的所有非bias部分设置requires_grad=False
# num_param = 0
# for name, param in model.named_parameters():
#     if "bias" not in name:
#         param.requires_grad = False
#     else:
#         num_param += param.numel()

# Soft Prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
# config
# Hard Prompt
from peft import PromptTuningConfig, PromptTuningInit
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是一段人与机器人的对话。",
                            num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话。")["input_ids"]),
                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")

from peft import PromptEncoderConfig, PromptEncoderReparameterizationType
config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10,
                            encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
                            encoder_dropout=0.1, encoder_num_layers=5, encoder_hidden_size=1024)

from peft import PrefixTuningConfig
config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=True)

from peft import LoraConfig
config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=".*\.1.*query_key_value", modules_to_save=["word_embeddings"])

from peft import IA3Config
config = IA3Config(task_type=TaskType.CAUSAL_LM)

# 创建模型
model = get_peft_model(model, config)

model.print_trainable_parameters()

In [ ]:
# 配置训练参数
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=20,
)

In [ ]:
# 创建训练器开始训练
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenizer_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()

# model.save_pretrained('./chatbot/peft.save_pretrained')  训练参数中已包含输出路径

In [ ]:
# 加载peft_model
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model=model, model_id="./chatbot/checkpoint-20/")

In [ ]:
# 测试模型性能
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

In [ ]:
# lora模型合并
merge_model = peft_model.merge_and_unload()

# 完整模型保存
merge_model.save_pretrained("./chatbot/merge_model")